# Waveforms to Xarray

<div class="alert alert-warning">

**Warning**: This part of obsplus is still very experimental and subject to rapid changes, proceed with caution.

</div>


The [xarray library](http://xarray.pydata.org/en/stable/) offers pandas-like data structures that are not limited to 2 dimensions (rows and columns). We have found working with seismic waveform data in such a way can be useful, but certainly is not as general as [obspy streams](https://docs.obspy.org/packages/autogen/obspy.core.stream.Stream.html). Particularly, Xarray data structures don't work well with gappy data or data with non-uniform sampling rates.

Before attempting to use these features in obsplus we highly recommend you read through the [xarray documentation](http://xarray.pydata.org/en/stable/) as the API may take a bit of time to learn. 

## Creating data arrays

Creating DataArrays from ObsPy objects is straight-forward. A single `Trace`, `Stream`, or collection (list-like) of either, or a mapping (dict-like) of either are all valid inputs. If a mapping is used, they keys will often correspond to an event id.

Conceptually the DataArray looks like this:

<img src="../../images/data_array2.png" width="350" height="350" align="left"/>
<br clear="all">

Each `DataArray` instance created by ObsPlus has three dimensions:

1. __stream_id__: The keys used in the dictionary or an integer starting at 0.

2. __seed_id__: The seed id (ie network.station.location.channel) of each trace.

3. __time__: Floating point values beginning at zero and incrementing by the sampling period.


In [ ]:
import obspy
import obsplus
from obsplus import obspy_to_array

In [ ]:
st = obspy.read()
# create data array from a trace
from_trace = obsplus.obspy_to_array(st[0])
# create data array from stream
from_stream = obsplus.obspy_to_array(st)
# create a data array from a list of streams
st_list = [st.copy() for _ in range(3)]
from_list = obsplus.obspy_to_array(st_list)
# create data array from a dict of streams
st_dict = {f'event{x}': st.copy() for x in range(3)}
from_dict = obsplus.obspy_to_array(st_dict)

In [ ]:
print(from_trace.dims)

The data array created from a single trace will only have a size of one in the stream_id and seed_id columns, and the time dimension will be as long as the trace.

In [ ]:
print(from_trace.shape)

The dataarray created from a dict of streams, however, will be larger:

In [ ]:
print(from_dict.shape)

The xarray `str` representation is fairly large, but very descriptive:

In [ ]:
# print value for the seed_id dimension
print(from_dict)

DataArrays can also be converted back to dictionaries of `Stream` objects. The transformation *should* be lossless:

In [ ]:
print(from_dict.ops.to_stream())

## Advantages of the DataArray

The DataArray has two potential advantages over the Stream representation:

   1. Efficiency 
    
   2. Organization

### Efficiency

There have been many efforts to improve efficiency of numpy/scipy functionality. Some of these, such as [Intel's MKL](https://software.intel.com/en-us/mkl) ship with scientific python distributions, like [Anaconda](https://www.anaconda.com/). These optimizations are great because you don't need to change anything about your code; it just runs faster.

Some of these optimizations involve making better use of modern hardware, particularly processors with many cores. It is much better to let the well-tested low-level libraries handle parallelism rather than implementing messy multiprocessing/multithreading python code when possible. For example, let's compare the time required to calculating FFTs for each `Trace` in a large `Stream` vs doing it all at once on a `DataArray` created with ObsPlus, both of which should return the same result. The latter will be more efficient because it allows numpy to better plan optimization strategies, as well as avoids python loops.

In [ ]:
import numpy as np
import xarray as xr

print(f'numpy version: {np.__version__}')
print(f'xarray version: {xr.__version__}')

In [ ]:
# create test streams with random data
import numpy as np
import obspy


num_stations = 200
sr = 100
data_length = sr * 60

traces = []
for station in ('{x:03d}'.format(x=x) for x in range(num_stations)):
    data = np.random.rand(data_length)
    stats = dict(network='OP', station=station, location='', channel='HHZ',
                 sampling_rate=sr)
    traces.append(obspy.Trace(data=data, header=stats))

st = obspy.Stream(traces=traces)
print(st)


In [ ]:
# convert to data array
dar = obspy_to_array(st)

In [ ]:
%%time
# Time looping through traces and performing fft
out1 = np.array([np.fft.rfft(tr.data) for tr in st])

In [ ]:
%%time
# Time performing fft in one-go on large numpy block
out2 = np.fft.rfft(dar.data, axis=-1)

In [ ]:
# after flattening one dimension in out2, the results should be (nearly) the same
out1 = np.array([np.fft.rfft(tr.data) for tr in st])
out2 = np.fft.rfft(dar.data, axis=-1)
assert np.allclose(out1, out2[0])

The xarray version is usually between 2 and 20 times faster, depending on the number of cores in your CPU and the python distribution you are using. However, this notebook may not show much of a difference if it was executed on the ReadTheDocs server. The best way to assess performance gains is to download and run this notebook yourself.

Moreover xarray provides ways of easily working with dask for distributed computing. This would be a bit more difficult using `Stream`s. See [this](http://xarray.pydata.org/en/stable/dask.html) for more details. 

### Organization

With the data organized in a 3D cube of sorts, it becomes fairly natural to slice and manipulate the data because xarray, like pandas, has intuitive and efficient indexing and sensible broadcasting. Here are a few examples of what you can do:

In [ ]:
# get middle 10 seconds of data
time_mean = dar.time.mean()
duration = dar.time.max() - dar.time.min()
dar.sel(time=slice(time_mean - 5, time_mean + 5))

In [ ]:
# Trim seed_id (channels) to only include data from every 13th station
stations = list('OP.{x:03d}..HHZ'.format(x=x) for x in range(0, data_length, 13))
dar.where(dar.seed_id.isin(stations), drop=True)

In [ ]:
# simple detrend using mean
dar - dar.mean(dim='time')

In [ ]:
# calculate rolling sta/lta on each channel
sta_samples = 25
lta_samples = 200

sta = dar.rolling(time=sta_samples).mean()
lta = dar.rolling(time=lta_samples).mean()

result = sta / lta
print(result.dropna(dim='time'))

### Obsplus Accessor methods
Obsplus registers an [xarray accessor](http://xarray.pydata.org/en/stable/internals.html#extending-xarray) to add seismic specific functionality. These are accessed via the `ops` attribute which is available as long as obsplus is imported. Here is a brief tour of some of the methods:

In [ ]:
# build a data array from the crandall dataset
import obsplus
ds = obsplus.load_dataset('crandall')
# get catalog, inventory and fetcher
cat = ds.event_client.get_events()
inv = ds.station_client.get_stations(network='TA')
fetcher = ds.get_fetcher()
# init dict of {event_id: stream} and get catalog/inventory
st_dict = dict(fetcher.yield_event_waveforms(time_before=5, time_after=30))
# create datarray
dar = obsplus.obspy_to_array_dict(st_dict)[40]

In [ ]:
print(cat)

In [ ]:
# select channels based on seed_ids (wildcards permitted)
filtered_dar = dar.ops.sel_sid('TA.*.*.BHZ')

# filter check
assert len(filtered_dar.seed_id)
for seed_id in filtered_dar.seed_id.values:
    assert seed_id.endswith('BHZ')
    assert seed_id.startswith('TA')

print(filtered_dar.seed_id)

In [ ]:
# Calcule rffts (note the "time" dimension has changed to "frequency")
rfft = dar.ops.rfft()
# print dimensions and corresponding size
print(dict(zip(rfft.dims, rfft.shape)))

In [ ]:
# Calculate irffts
irfft = rfft.ops.irfft()
# print dimensions and corresponding size
print(dict(zip(irfft.dims, irfft.shape)))

In [ ]:
# convert back into a dict of streams
st_dict = dar.ops.to_stream()
for event_id, st in st_dict.items():
    print(f'event_id: {event_id} stream_size: {len(st)}')

In [ ]:
# attach event information
dar_with_events = dar.ops.attach_events(cat)
# note the extra coords that have now been attached
print(dar_with_events.coords)

In [ ]:
# trim all waveforms to the mean of the P times picked on the same station
# if no such P pick exists the channels will not be trimmed
out = dar_with_events.ops.trim('p_time', aggregate_by='station')
print(out)

In [ ]:
# iterate over slices of stations
for seed_dar in dar.ops.iter_seed('station'):
    print(f'got channels: {set(seed_dar.seed_id.values)} in yielded data array')

In [ ]:
# calculate the norm of the data recorded at each station
from numpy.linalg import norm
dar.ops.agg(np.linalg.norm, level='station')